# Santander data preprocessing

In [ ]:
%load_ext kedro.extras.extensions.ipython

In [ ]:
%reload_kedro

In [ ]:
from typing import Iterator, Union, Tuple, List
from datetime import datetime

import pandas as pd
import numpy as np
import pprint
from kedro.extras.datasets.pandas import CSVDataSet
from kedro.io.core import get_filepath_str
from sklearn.model_selection import train_test_split

import recommender_gnn.pipelines.santander_preprocessing.nodes
from recommender_gnn.extras.datasets.chunks_dataset import (
 _load,
 _concat_chunks,
)
from recommender_gnn.pipelines.santander_preprocessing.nodes import(
    _stratify
)

pd.options.mode.chained_assignment = None
pd.set_option('expand_frame_repr', True)
pd.set_option("display.max_rows", 999)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

## Checking input dtypes and memory usage

In [ ]:
input_train_path = "santander_train_input"
input_test_path = "santander_test_input"

In [ ]:
train_input = _concat_chunks(context.catalog.load(input_train_path))

In [ ]:
test_input = _concat_chunks(context.catalog.load(input_test_path))

In [ ]:
train_input.loc[train_input.fecha_dato=='2015-11-28', :].shape

In [ ]:
train_input.drop_duplicates("ncodpers", keep='last').shape

In [ ]:
test_input.drop_duplicates("ncodpers", keep='last').shape

In [ ]:
train_shuffle = train_input.sample(frac=0.0001)

In [ ]:
train_shuffle.sort_values(by="fecha_dato").reset_index(drop=True).drop_duplicates("ncodpers", keep='last')

In [ ]:
customers = _stratify(train_shuffle, 30)

In [ ]:
train_shuffle.to_csv("/home/michal/projects/gid-ml-framework/recommender_gnn/src/tests/fixtures/csv/santander_test_sample.csv")

In [ ]:
train_input.info()

In [ ]:
train_input.memory_usage(deep=True)

In [ ]:
train_input.isnull()

## Comparing cleaned train and test dataframes

In [ ]:
cleaned_train_path = "santander_preprocessing_train.santander_cleaned"
cleaned_test_path = "santander_preprocessing_test.santander_cleaned"

In [ ]:
cleaned_train = _concat_chunks(context.catalog.load(cleaned_train_path))
cleaned_test = _concat_chunks(context.catalog.load(cleaned_test_path))

In [ ]:
lasts = cleaned_train.sort_values(["ncodpers", "fecha_dato"]).groupby("ncodpers").last()

In [ ]:
lasts.fecha_dato.unique()

In [ ]:
cleaned_train.isnull().sum()

In [ ]:
np.unique(cleaned_train.loc[:, "renta"])

In [ ]:
np.unique(cleaned_test.loc[:, "renta"])

In [ ]:
df = cleaned_train.copy()
mask = df['renta'].map(lambda x: isinstance(x, (int, float)))
df.loc[:, 'renta'] = df.loc[:, 'renta'].where(mask)
np.unique(df.loc[:, "renta"])

In [ ]:
df = cleaned_test
pd.to_numeric(df.loc[:, 'renta'], downcast="float", errors='coerce')

## Comparing final dataframes

In [ ]:
train_path = "santander_train"
val_path = "santander_val"
test_path = "santander_test"

In [ ]:
train = _concat_chunks(context.catalog.load(train_path))
val = _concat_chunks(context.catalog.load(val_path))
test = _concat_chunks(context.catalog.load(test_path))

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
val.head()

In [ ]:
val.dtypes

In [ ]:
test.head()

In [ ]:
test.dtypes

In [ ]:
test.dtypes == train.loc[:, test.columns].dtypes

In [ ]:
test.dtypes == val.loc[:, test.columns].dtypes